**test**

In [1]:
import os ; import pathlib as pl ; import pandas as pd ; import numpy as np ;  import matplotlib.pyplot as plt ; from pathlib import Path  
from flopy.plot.styles import styles ; import shapefile as shp ; import matplotlib as mpl ; import flopy ; import flopy.utils.binaryfile as bf 
from modflow_devtools.misc import get_env, timed ; write=get_env("WRITE", True);run=get_env("RUN", True) ; from flopy.utils.lgrutil import Lgr 
sim_ws    = Path("./99_test")                       ; sim_ws.mkdir(exist_ok=True)     ; sim_name = modelname= 'ex-gwf-lgrv-gc'
exe_name  = 'E:\\15_REPOS\\00_BETAMI\\bin\\mf6.exe'  ; workspace = figs_path = sim_ws       

In [2]:
length_units = "meters" ; time_units = "seconds"    ; k11_str = "5.e-07, 1.e-06, 5.e-05" ; nouter = 50 ; ninner = 100 ; hclose = 1e-6 ; rclose = 100.0
nper = 1 ;nlay = 25 ;nrow = 183 ;ncol = 147         ; nlaygc = 9 ;nrowcg = 61 ;ncolcg = 49 ;delr = 35 ;delc = 25 ;delv = 5 ;delrgc = 105 
delcgc = 75 ;delvgc = 15 ; top_str = "variable"     ; botm_str = "30 to -90" ; icelltype = 0 ; recharge = 0.0000000011098 ;

In [3]:
path = 'E:\\15_REPOS\\00_BETAMI\\bin\\data_modflow6-examples\\ex-gwf-lgrv'                 ; top = np.loadtxt(os.path.join(path,"top.dat"))
dt = [("k",int),("i",int),("j",int) ,("stage",float),("conductance",float),("rbot",float)] ; rivdat=np.loadtxt(os.path.join(path,"riv.dat"), dtype=dt)
ikzone = np.empty((nlay, nrow, ncol),dtype=float)
for k in range(nlay):                        
    fname = f"ikzone{k + 1}.dat"    ;f_ikzone = os.path.join(path, fname)                   ; ikzone[k, :, :] = np.loadtxt(f_ikzone)
a1 = pd.DataFrame(ikzone[24,:,:]) ; a1.to_csv (sim_ws/ "03_ikzone_xy.csv")                  ; print('ikzone',ikzone.shape)   # 01 02 03 *.csv
a1 = pd.DataFrame(top)    ; a1.to_csv (sim_ws/ "01_top.csv") ; a1 = pd.DataFrame(rivdat)    ; a1.to_csv (sim_ws/ "02_riv.csv")

ikzone (25, 183, 147)


In [4]:
modelname = 'child' 
def build_models(name,icoarsen=1,ncppl=None,rowcolspan=None,idomain=None,sim=None,modelname=None,xorigin=None,yorigin=None):

    nrowc, ncolc = coarsen_shape(icoarsen,nrow,ncol) ;delrc=delr*icoarsen ;delcc=delc*icoarsen  ; topc = array_resampler(top, icoarsen, "mean")
    if rowcolspan is not None:     istart    ,istop       ,jstart    ,jstop = rowcolspan        ;    nrowc = istop-istart    ;ncolc = jstop-jstart
    else:                          istart=0  ;istop=nrow  ;jstart=0  ;jstop = ncol
    topc = topc[istart: istop, jstart: jstop]
    # _______________________________________________________________________________________________________
    if ncppl is not None:
        nlayc = len(ncppl)         ; layer_index = [ncppl[0] - 1]
        for iln in ncppl[1:]:
            last = layer_index[-1] ; layer_index.append(iln + last)
    else:
        nlayc = nlay               ; layer_index = list(range(nlayc))
    # _______________________________________________________________________________________________________
    k11c = []
    for k in range(nlayc):
        ilay = layer_index[k]   ; a = array_resampler(k11[ilay], icoarsen, "maximum")  ; k11c.append(a[istart:istop, jstart:jstop])
    strt = nlayc * [topc]       ; head_filerecord = f"{modelname}.hds"     ; budget_filerecord = f"{modelname}.cbc"  
    # _______________________________________________________________________________________________________
    gwf = flopy.mf6.ModflowGwf     (sim, modelname = modelname, save_flows=True)       
    if idomain is None: idomain = 1
    flopy.mf6.ModflowGwfdis  (gwf,length_units=length_units,nlay=nlayc,nrow=nrowc,ncol=ncolc,
                              delr=delrc,delc=delcc,top=topc,botm=botm[layer_index],idomain=idomain,xorigin=xorigin,yorigin=yorigin)    
    flopy.mf6.ModflowGwfnpf  (gwf,k33overk=True,icelltype=icelltype,k=k11c,save_specific_discharge=True,k33=1.0)
    flopy.mf6.ModflowGwfic   (gwf,strt=strt);i2=gwf.dis.idomain.array ;rivdatc=riv_resample(icoarsen,nrow,ncol,rivdat,i2,rowcolspan);riv_spd={0:rivdatc}
    flopy.mf6.ModflowGwfriv  (gwf,stress_period_data=riv_spd,pname="RIV",)
    flopy.mf6.ModflowGwfrcha (gwf, recharge=recharge, pname="RCH")
    flopy.mf6.ModflowGwfoc   (gwf,head_filerecord=head_filerecord,budget_filerecord=budget_filerecord,saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")])
    return sim